# fonction fichier dico => csv 
# fonction lecture du dico_bresil (class Lecture_data_bresil)

In [33]:
import time

from Lecture_data_bresil import Lecture_data_bresil 

from pprint import PrettyPrinter 
def P (stuff , pp = PrettyPrinter(indent=4)) :
    return pp.pprint(stuff)


class  Ecriture_fichier_bresil () :
    
    def __init__ (self,path_file, dico, sep = '|') :
        self.sep = sep
        self.dico = dico
        self.file = open(path_file, 'w')
        
        liste_cle = [cle for cle in dico.keys()]
        self.liste_cle = [cle for cle in dico [liste_cle [0]].keys()]
        self.erreur = {}
        
        #print ('len liste_cle : ', len (self.liste_cle))
        
        
        
    def creation_ligne (self, enreg) :
        liste = []
        
        for cle in self.liste_cle :
            try:
                r = str(enreg [cle])
            except :
                r = ''
                try:
                    self.erreur [cle] += 1
                except:
                    self.erreur [cle] = 1
                
            r = r.replace ('|', '').replace ('\n', '')
            liste.append(r)
            continue
        
        li  = self.sep.join(liste)
        
        data = li.split(self.sep)
        if len (data) != len (self.liste_cle) :
            print ('erreur : ')
            P(enreg)
        
        return li
    
    def run (self,) :
        
        for cle in self.dico :
            enreg = self.dico [cle]
            ligne = self.creation_ligne (enreg)
                               
            data = ligne.split (self.sep)
            if len (data) != len(self.liste_cle) :
                print ('erreur', ligne)
                break
                
            i = ligne.find ('\r\n')
            if i != -1 :
                print ('erreur retour ', ligne)
                break
                
            li = ligne + '\n'
            self.file.write (li)
            continue
        
        self.file.close()
        if len(self.erreur) != 0 :
            print ('erreur')
            P(self.erreur)
            print ()
        return

def lecture_data_bresil ():
    
    L = Lecture_data_bresil ()
       
    dico_resultat = L.dico_customers ()
    
    assert L.nombre_erreur == 2765
    
    assert len(dico_resultat) == 99163
    return dico_resultat



# test ecriture fichier dico => csv   

In [34]:

def test_ecriture_fichier_bresil () :
    dico  = {'1': {"e": 0, "f" : 1},
             '2' : {"e": 2, "f" : 3} , 
            '3' :  {"e": 'e||e', "f" : 'f\n\nf'}}
    path_file = './test.txt'
    E = Ecriture_fichier_bresil (path_file, dico)
    E.run ()
    
    f = open(path_file , 'r')
    resultat = []
    while (True) :
        li = f.readline ()
        if li == '' :
            break
        resultat.append(li [:-1])
        data = li [:-1].split ('|')
        if len (data) != 2 :
            print ('erreur', li)
        continue
        
    
    
    voulu = ['0|1', '2|3', 'ee|ff']
    assert resultat == voulu
    

if __name__ == '__main__' :
    t = time.time()
    test_ecriture_fichier_bresil ()
    print ('fin test_ecriture_fichier_bresil en ', time.time () - t)    

fin test_ecriture_fichier_bresil en  0.0008680820465087891


# creation fichier csv bresil

In [35]:
dico = lecture_data_bresil ()

#print ('len(dico) = ', len(dico) )

path_file = './bresil.csv'
E = Ecriture_fichier_bresil (path_file, dico)
E.run ()

assert len(dico) == 99163
assert len (E.liste_cle) == 40





# test creation './bresil.csv'

In [36]:
path_file = './bresil.csv'
#path_file = '../data/#test/data_alimentation/evenements.txt'

f = open( path_file, 'r', newline='\n')

taille = 40

nombre = 0

while( True) :
    li = f.readline ()
    if li == '' :
        break
    nombre += 1
    data = li [:-1].split ('|')
    if len (data) != taille :
        print ('taille ?', len(data))
        print (li)
        
        break
        
        
    continue
    
#print ()
#print (nombre)
assert nombre == 99163


# liste des formats detectés par le dataframe

# liste format date
## https://help.talend.com/r/6K8Ti_j8LkR03kjthAW6fg/~wDyssNBFPIG2jgx3fux3Q

In [37]:
path_file = './bresil.csv'
nombre_colonne = len(liste_cle)
columns = [str(i) for i in range(0, nombre_colonne)]
df = pd.read_csv(path_file, sep = '|',
                 lineterminator = '\n',
                 names = columns,
                 infer_datetime_format = True,
                )

liste_type = df.dtypes.tolist()
assert len(liste_type) == 40

liste_format = []
liste_valeur = df.loc[0,].tolist()
position = 0
format_valeur = {}
for v_type in liste_type :
    v_type = str(v_type)
    if v_type == 'int64' or v_type == 'float64' :
        v_type= 'numeric'
    else:
        assert v_type == 'object'
    
    print (str(position), ' = ',v_type, ' pour ', liste_valeur [position])
    
    format_valeur [str (position)] = [v_type, liste_valeur [position]]
    position += 1
    continue

0  =  object  pour  06b8999e2fba1a1fbc88172c00ba8bc7
1  =  object  pour  861eff4711a542e4b93843c6dd7febb0
2  =  numeric  pour  14409
3  =  object  pour  franca
4  =  object  pour  SP
5  =  numeric  pour  -20.500252601080785
6  =  numeric  pour  -47.40036721880004
7  =  object  pour  SP
8  =  object  pour  00e7ee1b050b8499577073aeb2a297a1
9  =  object  pour  delivered
10  =  object  pour  2017-05-16 15:05:35
11  =  object  pour  2017-05-16 15:22:12
12  =  object  pour  2017-05-23 10:47:57
13  =  object  pour  2017-05-25 10:35:35
14  =  object  pour  2017-06-05 00:00:00
15  =  object  pour  a9516a079e37a9c9c36b9b78b10169e8
16  =  object  pour  7c67e1448b00f6e969d365cea6b010ab
17  =  object  pour  2017-05-22 15:22:12
18  =  numeric  pour  124.99
19  =  numeric  pour  21.88
20  =  object  pour  moveis_escritorio
21  =  numeric  pour  41.0
22  =  numeric  pour  1141.0
23  =  numeric  pour  1.0
24  =  numeric  pour  8683.0
25  =  numeric  pour  54.0
26  =  numeric  pour  64.0
27  =  numeric 

# calcul des dico_position et dico_type
# creation Zone ID et Zone date_evenement


In [40]:
#P (liste_cle)
    
dico_position = {}
dico_position_nom_zone = {}

i = 0 
for nom_zone in liste_cle :
    dico_position [nom_zone] = i
    dico_position_nom_zone [str(position)] = nom_zone
    i += 1
    continue
 
position_bresil_voulu = {   'customer_city': 3,
    'customer_id': 0,
    'customer_state': 4,
    'customer_unique_id': 1,
    'customer_zip_code_prefix': 2,
    'freight_value': 19,
    'geolocation_lat': 5,
    'geolocation_lng': 6,
    'geolocation_state': 7,
    'order_approved_at': 11,
    'order_delivered_carrier_date': 12,
    'order_delivered_customer_date': 13,
    'order_estimated_delivery_date': 14,
    'order_id': 8,
    'order_purchase_timestamp': 10,
    'order_status': 9,
    'payment_installments': 33,
    'payment_sequential': 31,
    'payment_type': 32,
    'payment_value': 34,
    'price': 18,
    'product_category_name': 20,
    'product_category_name_english': 28,
    'product_description_lenght': 22,
    'product_height_cm': 26,
    'product_id': 15,
    'product_length_cm': 25,
    'product_name_lenght': 21,
    'product_photos_qty': 23,
    'product_weight_g': 24,
    'product_width_cm': 27,
    'review_answer_timestamp': 39,
    'review_comment_message': 37,
    'review_comment_title': 36,
    'review_creation_date': 38,
    'review_score': 35,
    'seller_city': 29,
    'seller_id': 16,
    'seller_state': 30,
    'shipping_limit_date': 17}

assert position_bresil_voulu == dico_position   
    



liste_position_fausse = [36, 37, 15, 15, 0, 1]
liste_position_date = [10, 11, 12, 13, 14, 37, 38, 39]
dico_type = {}
position = 0
format_date = '%Y-%m-%d %H:%M:%S'

for nom_zone in liste_cle :
        
    if position in liste_position_fausse :
        del dico_position [nom_zone]
        position += 1
        continue
    if position in liste_position_date :
        dico_type [nom_zone] = {'format' : format_date , 'type' : 'date'}
        position += 1
        continue
    dico_type [nom_zone] = {'format' : 'standard' , 'type' : 'string'}
    position += 1
    continue
    
    

position = dico_position ['customer_city']
position_bresil ['ID'] = position
del dico_position ['customer_city']
del dico_type ['customer_city']

position = dico_position ['order_purchase_timestamp']
position_bresil ['date_evenement'] = position
del dico_position ['order_purchase_timestamp']
del dico_type ['order_purchase_timestamp']


In [42]:
#P(dico_type)

# creation des parametres dico_evenements (position et type)

In [44]:
import sys, json

path = "../outils"
if path not in sys.path :
    sys.path.append (path)
    
from PARAMETRAGES import Update_create__environnement


nom_environnement = 'bresil'
dico_evenements = Update_create__environnement (nom_environnement,
                                                v_type = dico_type,
                                                position = dico_position)

nom_environnement = "#test"        
path = '../data/'+ nom_environnement + '/parametres/'
nameFile = "dico_evenements_2.json"
pathFile = path + nameFile
f = open (pathFile, "r")
string = f.read ()
dico_evenements = json.loads(string)
P (dico_evenements)



{   'alimentation': {   'execution': {   'date_debut_alimentation': '2021-02-01 '
                                                                    '00:00:00',
                                         'dico_ajout': {},
                                         'index_data': 'data_fake',
                                         'nombre_erreur_max': 0,
                                         'taille_globale': [0, 0]}},
    'apprentissage': {'liste_execution': []},
    'creation_dictionnaire': {   'ID': {'parametres': [], 'travail': None},
                                 'date_evenement': {   'parametres': [],
                                                       'travail': 'date'},
                                 'description': {   'parametres': [   {   'format': 'standard'}],
                                                    'travail': 'analyse_mot'},
                                 'nomenclature_1': {   'parametres': [   {   'format': 'standard'}],
                             

# creation fichier par mois

In [52]:
from datetime import datetime
import time, os


def filtrage_copie (mois, position_bresil,
                    path_file = './bresil.csv',
                    path_ecriture = './mois_test.json',
                    format_date = '%Y-%m-%d %H:%M:%S' ) :
    
    
    position = posv_type, valeur = ition_bresil ['date_evenement']
    f = open( path_file, 'r', newline='\r\n')
    g = open (path_ecriture, 'w')
    
    nombre = 0
    
    while ( True) :
        li = f.readline ()
        if li == '' :
            break
        
        data = li [:-1].split ('|')
        date_evenement = data [position]
        
        date_time_obj = datetime. strptime(date_evenement, format_date)
        
        if date_time_obj.month == mois :
            g.write (li)
            nombre += 1
        
        
        continue
    g.close()
    return nombre

        
if __name__ == '__main__' :
    t = time.time()
    mois = 1
    path_ecriture = './mois_test.csv'
    n = filtrage_copie (mois, position_bresil,
                        path_file = './bresil.csv',
                        path_ecriture = path_ecriture)
    
    assert n == 8055
    os.remove (path_ecriture)
    #print ('nombre sur ce mois =', n)
    print ('fin test_ecriture_fichier_bresil en ', time.time () - t)  

fin test_ecriture_fichier_bresil en  0.8154058456420898


# ecriture fichier par mois

In [54]:
import sys

path = "../outils"
if path not in sys.path :
    sys.path.append (path)
from validationPath import validationPath

Path_ecriture = '../data/#bresil/data_alimentation/'
validationPath (Path_ecriture)

for mois in range(1, 13) :
    path_ecriture = Path_ecriture + 'mois_'+ str(mois) + '.csv'
    n = filtrage_copie (mois, position_bresil,
                        path_file = './bresil.csv', 
                        path_ecriture = path_ecriture,)
    print ('ecriture de '+ str(n) + ' lignes  dans '+path_ecriture)
    
    
    
print ('end_of_job')

ecriture de 8055 lignes  dans ../data/#bresil/data_alimentation/mois_1.csv
ecriture de 8483 lignes  dans ../data/#bresil/data_alimentation/mois_2.csv
ecriture de 9863 lignes  dans ../data/#bresil/data_alimentation/mois_3.csv
ecriture de 9316 lignes  dans ../data/#bresil/data_alimentation/mois_4.csv
ecriture de 10544 lignes  dans ../data/#bresil/data_alimentation/mois_5.csv
ecriture de 9383 lignes  dans ../data/#bresil/data_alimentation/mois_6.csv
ecriture de 10292 lignes  dans ../data/#bresil/data_alimentation/mois_7.csv
ecriture de 10810 lignes  dans ../data/#bresil/data_alimentation/mois_8.csv
ecriture de 4293 lignes  dans ../data/#bresil/data_alimentation/mois_9.csv
ecriture de 4945 lignes  dans ../data/#bresil/data_alimentation/mois_10.csv
ecriture de 7521 lignes  dans ../data/#bresil/data_alimentation/mois_11.csv
ecriture de 5658 lignes  dans ../data/#bresil/data_alimentation/mois_12.csv
end_of_job


# creation des fichiers par mois 
